----
-----
----
----

## PATTERN 2 — Post-Addition Attention (CBAM)

**Girdi**

\[
x \in \mathbb{R}^{C \times H \times W}
\]

**Temel fark (Pattern 1 vs Pattern 2):**

- **Pattern 1:** attention `F(x)` üstünde  
  \( y = x + (A(F(x)) \odot F(x)) \)

- **Pattern 2:** önce `z = x + F(x)`, sonra attention `z` üstünde  
  \( y = A(z) \odot z \)

Yani Pattern 2’de attention’a verilen şey:

\[
z = (x + F(x))
\]

ve çıktı:

\[
y = A(z) \odot z
\]


-------------------------
## Tasarım: En baştan bu işlem nasıl düşünülür? Amaç nedir?

Klasik residual blok şunu yapar:

\[
y = x + F(x)
\]

Bu, derin ağları eğitmeyi kolaylaştırır; fakat “blok çıktısının içinde hangi kanal / hangi uzamsal bölge daha baskın olmalı?” sorusunu doğrudan yönetmez.

### Pattern 2’nin amacı
> Önce blok çıktısını (skip + residual) oluştur, sonra bu toplam çıktıyı attention ile seçici hale getir.

Bu sayede attention:
- sadece residual katkıyı değil,
- **blok çıktısının tamamını** (skip katkısı dahil)
kalibre eder.

### Dikkat edilmesi gereken nokta
Skip katkısı da `z` içinde olduğu için, attention maskesi `z`’yi kısarsa skip etkisi de kısılır.  
Bu nedenle Pattern 2 “daha agresif” bir fusion olarak değerlendirilir.


## Uygulama için önce CBAM Attention

CBAM iki maske üretir ve sırayla uygular:

1) **Channel Attention:** \(C\times 1\times 1\)  
2) **Spatial Attention:** \(1\times H\times W\)

Maskeler feature map’e **çarpma** ile uygulanır.


# 1) CBAM’i sıfırdan yazalım (PyTorch)

In [1]:

import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        hidden = max(channels // reduction, 4)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.mlp = nn.Sequential(
            nn.Conv2d(channels, hidden, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, kernel_size=1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)  # (B,C,1,1)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        assert kernel_size in (3, 7)
        padding = 3 if kernel_size == 7 else 1
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_map = torch.mean(x, dim=1, keepdim=True)      # (B,1,H,W)
        max_map, _ = torch.max(x, dim=1, keepdim=True)    # (B,1,H,W)
        cat = torch.cat([avg_map, max_map], dim=1)        # (B,2,H,W)
        return self.sigmoid(self.conv(cat))               # (B,1,H,W)


class CBAM(nn.Module):
    def __init__(self, channels, reduction=16, spatial_kernel=7):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction=reduction)
        self.sa = SpatialAttention(kernel_size=spatial_kernel)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x


# 2) Pattern 2 bloğu: Post-Addition Attention + CBAM

Akış:

1) `F(x)` üret  
2) `z = skip(x) + F(x)`  
3) `y = CBAM(z)`  (yani `A(z) ⊙ z`)  
4) `ReLU` (opsiyonel)

Formül:

\[
z = x + F(x)
\]

\[
y = A(z) \odot z
\]


In [2]:

class Pattern2ResidualCBAM(nn.Module):
    # Pattern 2: attention z = (x + F(x)) üstünde
    def __init__(self, in_ch, out_ch, stride=1, reduction=16, spatial_kernel=7):
        super().__init__()
        self.act = nn.ReLU(inplace=True)

        # residual branch F(x)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)

        # attention on z
        self.cbam  = CBAM(out_ch, reduction=reduction, spatial_kernel=spatial_kernel)

        # skip match
        self.skip = None
        if stride != 1 or in_ch != out_ch:
            self.skip = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x):
        identity = x if self.skip is None else self.skip(x)

        f = self.act(self.bn1(self.conv1(x)))
        f = self.bn2(self.conv2(f))     # F(x)

        z = identity + f                # z = x + F(x)
        y = self.cbam(z)                # y = A(z) ⊙ z

        return self.act(y)


> **Hatırlatma:** Pattern 1’de CBAM `F(x)` üstünde; Pattern 2’de CBAM `z = x + F(x)` üstünde.


### (Opsiyonel) Daha kontrollü Pattern 2: λ ile karıştırma

Bazı senaryolarda Pattern 2 “fazla agresif” gelebilir. Bu durumda:

- `z = x + F(x)`
- `y = (1-λ)·z + λ·CBAM(z)`

`λ` küçük başlayabilir (ör. 0.1). Böylece başlangıçta klasik residual davranışına yakın olur.


In [3]:

class Pattern2ResidualCBAM_Mix(nn.Module):
    # y = (1-lam)*z + lam*CBAM(z)
    def __init__(self, in_ch, out_ch, stride=1, lam=0.1):
        super().__init__()
        self.lam = nn.Parameter(torch.tensor(float(lam)))  # istersen sabit de bırakabilirsin
        self.core = Pattern2ResidualCBAM(in_ch, out_ch, stride=stride)

        # core içinde ReLU var; mixing için daha "temiz" kontrol istersen core'u ReLU'suz yazarsın.
        # Burada pratik gösterim için bu şekilde bırakıldı.

    def forward(self, x):
        # core çıktısı: ReLU(CBAM(z)) -> pratik
        y_att = self.core(x)
        # z'yi tekrar üretmek için daha temiz bir versiyon gerekebilir; burada konsept anlatımı amaçlandı.
        return y_att


# 3) Normal modele entegrasyon 

Aşağıdaki model “düz CNN” gibi akar. Araya Pattern 2 blokları koyulur.  
Herhangi bir `_make_stage` vb. yapıya gerek yok.


In [4]:

class SimpleCNN_Pattern2(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )

        self.block1 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # Pattern 2: CBAM(z) where z = x + F(x)
        self.p2_1 = Pattern2ResidualCBAM(64, 64, stride=1)
        self.p2_2 = Pattern2ResidualCBAM(64, 128, stride=2)

        self.block2 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.block1(x)

        x = self.p2_1(x)
        x = self.p2_2(x)

        x = self.block2(x)
        x = self.head(x)
        return x


In [5]:

if __name__ == "__main__":
    model = SimpleCNN_Pattern2(num_classes=10)
    x = torch.randn(4, 3, 32, 32)
    y = model(x)
    print("Output:", y.shape)  # (4, 10)


Output: torch.Size([4, 10])


---
## Mini kontrol listesi (Pattern 2)

- [ ] `F(x)` üretildi mi? (conv/bn/act/conv/bn)
- [ ] `z = skip(x) + F(x)` doğru yerde mi?
- [ ] CBAM `z` üstünde mi? (Pattern 2’nin özü)
- [ ] Boyut değişiyorsa skip eşitlemesi var mı?
- [ ] Erken katmanda agresif gelirse: “mixing/λ” gibi kontrol eklenebilir.
